In [351]:
%reset -sf

In [352]:
#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

In [353]:
strr = """be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce"""

In [354]:
with open("/Users/hkmac/Downloads/input.txt") as f:
    strr = f.read()

In [355]:
inp = [row.split(" | ") for row in strr.strip().split("\n")]

In [356]:
left = [[tuple(sorted(list(x))) for x in row[0].split()] for row in inp]

In [357]:
right = [[tuple(sorted(list(x))) for x in row[1].split()] for row in inp]

In [358]:
ref = ["abcefg", "cf", "acdeg", "acdfg", "bcdf",
       "abdfg", "abdfeg", "acf", "abcdefg", "abcdfg"]
ref = [tuple(sorted(list(x))) for x in ref]

idx = {word:i for i,word in enumerate(ref)}

ref.sort()
ref

[('a', 'b', 'c', 'd', 'e', 'f', 'g'),
 ('a', 'b', 'c', 'd', 'f', 'g'),
 ('a', 'b', 'c', 'e', 'f', 'g'),
 ('a', 'b', 'd', 'e', 'f', 'g'),
 ('a', 'b', 'd', 'f', 'g'),
 ('a', 'c', 'd', 'e', 'g'),
 ('a', 'c', 'd', 'f', 'g'),
 ('a', 'c', 'f'),
 ('b', 'c', 'd', 'f'),
 ('c', 'f')]

In [359]:
arr = left[0]
brr = right[0]

In [360]:
arr

[('a', 'b', 'c', 'd', 'e', 'f', 'g'),
 ('a', 'c', 'g'),
 ('a', 'd', 'e', 'f', 'g'),
 ('c', 'd', 'f', 'g'),
 ('a', 'c', 'd', 'e', 'f', 'g'),
 ('a', 'c', 'e', 'f', 'g'),
 ('a', 'b', 'd', 'e', 'f', 'g'),
 ('a', 'b', 'c', 'd', 'e', 'g'),
 ('c', 'g'),
 ('a', 'b', 'c', 'e', 'f')]

In [361]:
brr

[('a', 'b', 'c', 'd', 'e', 'f', 'g'),
 ('c', 'd', 'f', 'g'),
 ('c', 'g'),
 ('a', 'c', 'g')]

In [362]:
res = 0
for arr, brr in zip(left, right):

    abc = "abcdefg"
    mappings = itertools.permutations("abcdefg", r=7)
    for mapping in mappings:
        mapping_inv = {x:abc[i] for i,x in enumerate(mapping)}
        mapping = {abc[i]:x for i,x in enumerate(mapping)}

        xrr = [tuple(sorted(mapping_inv[x] for x in word)) for word in arr]
        xrr.sort()
        if xrr == ref:
            print("ok")
            break
            
    yrr = [idx[tuple(sorted(mapping_inv[x] for x in word))] for word in brr]
    yrr = int("".join((str(x)) for x in yrr))
    res += yrr

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok


In [363]:
res

1012272

In [364]:
ref

[('a', 'b', 'c', 'd', 'e', 'f', 'g'),
 ('a', 'b', 'c', 'd', 'f', 'g'),
 ('a', 'b', 'c', 'e', 'f', 'g'),
 ('a', 'b', 'd', 'e', 'f', 'g'),
 ('a', 'b', 'd', 'f', 'g'),
 ('a', 'c', 'd', 'e', 'g'),
 ('a', 'c', 'd', 'f', 'g'),
 ('a', 'c', 'f'),
 ('b', 'c', 'd', 'f'),
 ('c', 'f')]

In [346]:
xrr

[('a', 'b', 'c', 'd', 'e', 'f', 'g'),
 ('a', 'b', 'c', 'd', 'f', 'g'),
 ('a', 'b', 'c', 'e', 'f', 'g'),
 ('a', 'b', 'd', 'e', 'f', 'g'),
 ('a', 'b', 'd', 'f', 'g'),
 ('a', 'c', 'd', 'e', 'g'),
 ('a', 'c', 'd', 'f', 'g'),
 ('a', 'c', 'f'),
 ('b', 'c', 'd', 'f'),
 ('c', 'f')]

[8, 3, 9, 4]

In [317]:
brr

[('a', 'b', 'c', 'd', 'e', 'f', 'g'),
 ('b', 'c', 'd', 'e', 'f'),
 ('b', 'c', 'd', 'e', 'f', 'g'),
 ('b', 'c', 'e', 'g')]